In [ ]:
import zipfile
import os

zip_path = r"C:\Users\Adarsh\OneDrive\Desktop\Fracture_Detector-main\FINAL_DATASET.zip"
extract_to = "/content/main"

if zipfile.is_zipfile(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("Extraction complete.")
else:
    print("The file is not a valid zip file.")

: 

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from tensorflow.keras.preprocessing import image

Set dataset paths


In [13]:
train_dir = "/content/main/data/train"
val_dir = "/content/main/data/val"
img_size = (150, 150)
batch_size = 32

Data generators

In [14]:
train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


Build CNN model

In [15]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

c:\Users\Adarsh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', save_best_only=True, mode='min')


In [18]:
model.fit(train_gen, validation_data=val_gen, epochs=5, callbacks=[checkpoint])

c:\Users\Adarsh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - accuracy: 0.6737 - loss: 0.6238

277/277 ━━━━━━━━━━━━━━━━━━━━ 130s 466ms/step - accuracy: 0.6740 - loss: 0.6233 - val_accuracy: 0.6583 - val_loss: 0.7129
Epoch 2/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 53s 189ms/step - accuracy: 0.9396 - loss: 0.1665 - val_accuracy: 0.7200 - val_loss: 0.9282
Epoch 3/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 51s 184ms/step - accuracy: 0.9881 - loss: 0.0427 - val_accuracy: 0.6583 - val_loss: 1.1130
Epoch 4/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 52s 188ms/step - accuracy: 0.9983 - loss: 0.0092 - val_accuracy: 0.6967 - val_loss: 1.5428
Epoch 5/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 52s 187ms/step - accuracy: 0.9983 - loss: 0.0092 - val_accuracy: 0.6383 - val_loss: 1.9064


In [22]:
img_path = r"C:\Users\Adarsh\OneDrive\Desktop\Project1\ravi_test.jpg" 
img = load_img(img_path, target_size=(150, 150))
img_array = img_to_array(img) / 255.0
input_data = np.expand_dims(img_array, axis=0)
some_input = np.expand_dims(img_array, axis=0)

In [23]:
pred = model.predict(some_input)
print("Raw prediction:", pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
Raw prediction: [[0.19969141]]


In [24]:
if pred[0][0] <= 0.5:
    print("Prediction: Class 0 (e.g., Fractured)")
else:
    print("Prediction: Class 1 (e.g., Normal)")

Prediction: Class 0 (e.g., Fractured)
